In [ ]:
import pandas as pd

In [ ]:
has_ans= pd.read_csv('has_answer_quac.csv')
no_ans= pd.read_csv('no_answer_quac.csv')

In [ ]:
has_ans.shape

In [ ]:
no_ans.shape

In [ ]:
has_ans= has_ans.drop(has_ans.columns[0], axis=1)
has_ans= has_ans.drop(has_ans.columns[8], axis=1)
no_ans= no_ans.drop(no_ans.columns[0], axis=1)
no_ans= no_ans.drop(no_ans.columns[7], axis=1)

In [ ]:
has_ans ['question']= has_ans ['question'].str.replace('[^\w\s]','')
no_ans ['question']= no_ans ['question'].str.replace('[^\w\s]','')
has_ans ['question']= has_ans ['question'].str.lower()
no_ans ['question']= no_ans ['question'].str.lower()

In [ ]:
has_ans['question_type']=''
keywords =["why", "when", "how", "what", "who", "where",'which','does']
for  (i, answer) in enumerate(has_ans['question']):
    words = answer.split()
    for key in keywords:
        if key in  words:
            has_ans['question_type'][i]=key


In [ ]:
for  (i, answer) in enumerate(has_ans['question_type']):
    if (has_ans['followup'][i]=='y' and answer=='' ):
        has_ans['question_type'][i]='followup_other'
    elif (answer=='') and (has_ans['followup'][i]!='y'):
        has_ans['question_type'][i]= 'other'

In [ ]:
has_ans.question_type.value_counts()

In [ ]:
has_ans.head()

<ul>
  <li>context - provided context</li>
  <li>id - ID of question</li>
  <li>TF - whether the prediction was correct or not (True/False)</li>
  <li>pred_answer - what was the predicted answer</li>
  <li>possible_answers - acceptable answers</li>
  <li>question - question ansked</li>
  <li>followup - whether the question was a followup question or not</li>
  <li>yesno - whether the question was yes/no question  </li>
</ul>

In [ ]:
no_ans['question_type']=''
keywords = ["why", "when", "how", "what", "who", "where",'which','does']
for  (i, answer) in enumerate(no_ans['question']):
    words = answer.split()
    for key in keywords:
        if key in words:
            no_ans['question_type'][i]=key
            

In [ ]:
for  (i, answer) in enumerate(no_ans['question_type']):
    if (no_ans['followup'][i]=='y' and answer=='' ):
        no_ans['question_type'][i]='followup_other'
    elif (answer=='') and (no_ans['followup'][i]!='y'):
        no_ans['question_type'][i]= 'other'


In [ ]:
no_ans.question_type.value_counts()

In [ ]:
no_ans.head()

<ul>
  <li>id - ID of question</li>
  <li>TF - whether the prediction was correct or not (True/False)</li>
  <li>answer - what was the predicted answer</li>
  <li>question - question ansked</li>
  <li>followup - whether the question was a followup question or not</li>
  <li>yesno - whether the question was yes/no question  </li>
  <li>context - provided context</li>
</ul>

In [ ]:
has_ans.TF.value_counts()

In [ ]:
no_ans.TF.value_counts()

Corresponds with the scores: 
<ul>
    <li> "exact": 46.81805819961925,</li>
    <li>"f1": 57.674530671382776, </li>
    <li>"total": 7354, </li>
    <li>"HasAns_exact": 35.71915473755965, </li>
    <li>"HasAns_f1": 49.3248975046608, </li>
    <li>"HasAns_total": 5868,</li>
    <li>"NoAns_exact": 90.64602960969044, </li>
    <li>"NoAns_f1": 90.64602960969044, </li>
    <li>"NoAns_total": 1486, </li>
    <li>"best_exact": 48.88496056567854,</li>
    <li>"best_exact_thresh": -1.9336259365081787, </li>
    <li>"best_f1": 62.67965908896002, </li>
    <li>"best_f1_thresh": -1.120149374008178</li>
 </ul>

In [ ]:
no_ans = no_ans.rename({'answer': 'pred_answer'}, axis='columns')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
def plot_histogram(data, label='',datalabel=''):
    plt.hist(data, bins=100, normed=1, alpha=1.0)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.title((datalabel+' '+label+' length').capitalize())
    plt.savefig('graphs/'+datalabel+'_length_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
plot_histogram([(len(str(a))) for a in has_ans['pred_answer']], 'answer','has_ans')
plot_histogram([q for q in has_ans['question'].str.len()], 'question','has_ans')
plot_histogram([c for c in has_ans['context'].str.len()], 'context','has_ans')

In [ ]:
has_ans['ans_length']  = has_ans['pred_answer'].str.len()
has_ans['con_length']  = has_ans['context'].str.len()
has_ans['que_length']  = has_ans['question'].str.len()

In [ ]:
def question_questiontype(dataset,label,x,datalabel):
    answers= []
    keywords = ["why", "when", "how", "what", "who", "where",'which','other','followup_other']
    for key in keywords:
        answers.append([len(answer) for (i, answer) in enumerate(dataset) if key in has_ans.iloc[i]['question_type'].lower()])
    bins = np.linspace(0, x, 15)
    plt.hist(answers, bins, normed=True, label=keywords)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.legend()
    plt.title((datalabel+' '+label+' questiontype').capitalize())
    plt.savefig('graphs/'+datalabel+'_questiontype_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
question_questiontype(has_ans['pred_answer'],'answer',100,'has_ans' )
question_questiontype(has_ans['context'],'context',3000,'has_ans')
question_questiontype(has_ans['question'],'question',150,'has_ans')

In [ ]:
incorrect=has_ans[has_ans['TF'] == False]
correct=has_ans[has_ans['TF']==True]

# Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','has_ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','has_ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','has_ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'has_ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'has_ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'has_ans_incorrect')

# Only correct answers


In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','has_ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','has_ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','has_ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'has_ans_correct' )
question_questiontype(correct['context'],'context',3000,'has_ans_correct')
question_questiontype(correct['question'],'question',150,'has_ans_correct')

In [ ]:
sns.histplot(data=has_ans, x="ans_length",bins=100, hue='TF',palette=["C0", "C1"])
plt.show()

## Pie charts


In [ ]:

def piechart (dataset,labels,name):
    values=dataset.value_counts()
    labels=dataset.value_counts().index.values
    plt.figure(figsize=(8,8))
    plt.pie(values,labels=labels,autopct='%1.1f%%',shadow=True)
    plt.legend(labels, bbox_to_anchor=(0.85,1.025), loc="upper left")
    plt.axis('equal')
    plt.tight_layout()
    plt.title((name+' question type distribution').capitalize())
    plt.savefig('graphs/'+name+'_questiontype_pie.png')
    plt.show()
    print(dataset.value_counts())



In [ ]:
labels=''
piechart(has_ans['question_type'],labels,'has_ans')
piechart(incorrect['question_type'],labels,'has_ans_incorrect')
piechart(correct['question_type'],labels,'has_ans_correct')

In [ ]:
has_ans['followup'].value_counts()

In [ ]:
correct['followup'].value_counts()

In [ ]:
def pie_in(labels,dataset,name,feature):
    plt.figure(figsize=(4,4))
    values=dataset.value_counts().values
    labels=dataset.value_counts().index.values
    plt.pie(values,labels=labels,autopct='%1.1f%%',shadow=True)
    plt.legend(labels)
    plt.axis('equal')
    plt.tight_layout()
    plt.title((name+' '+feature+' attribute distribution').capitalize())
    plt.savefig('graphs/'+name+'_'+feature+'_pie.png')
    plt.show()
    print(dataset.value_counts())

In [ ]:
labels=['y','m','n']
pie_in(labels,has_ans['followup'],'has_ans','followup')
pie_in(labels,incorrect['followup'],'has_ans_incorrect','followup')
pie_in(labels,correct['followup'],'has_ans_correct','followup')

In [ ]:
has_ans['yesno'].value_counts()

In [ ]:
labels=['y','x','n']
pie_in(labels,has_ans['yesno'],'has_ans','yesno')
pie_in(labels,incorrect['yesno'],'has_ans_incorrect','yesno')
pie_in(labels,correct['yesno'],'has_ans_correct','yesno')


# Unanswerable part

In [ ]:
plot_histogram([(len(str(a))) for a in no_ans['pred_answer']], 'answer','no_ans')
plot_histogram([q for q in no_ans['question'].str.len()], 'question','no_ans')
plot_histogram([c for c in no_ans['context'].str.len()], 'context','no_ans')

In [ ]:
no_ans['ans_length']  = no_ans['pred_answer'].str.len()
no_ans['con_length']  = no_ans['context'].str.len()
no_ans['que_length']  = no_ans['question'].str.len()

In [ ]:
def question_questiontype(dataset,label,x,datalabel):
    answers= []
    keywords = ["why", "when", "how", "what", "who", "where",'which','other','followup_other']
    for key in keywords:
        answers.append([len(answer) for (i, answer) in enumerate(dataset) if key in no_ans.iloc[i]['question_type'].lower()])
    bins = np.linspace(0, x, 15)
    plt.hist(answers, bins, normed=True, label=keywords)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.legend()
    plt.title((label+' questiontype').capitalize())
    plt.savefig('graphs/'+datalabel+'_questiontype_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
question_questiontype(no_ans['pred_answer'],'answer',100,'no_ans' )
question_questiontype(no_ans['context'],'context',3000,'no_ans')
question_questiontype(no_ans['question'],'question',150,'no_ans')

In [ ]:
incorrect=no_ans[no_ans['TF'] == False]
correct=no_ans[no_ans['TF']==True]

## Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','no_ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','no_ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','no_ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'no_ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'no_ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'no_ans_incorrect')

## Only correct answers

In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','no_ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','no_ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','no_ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'no_ans_correct' )
question_questiontype(correct['context'],'context',3000,'no_ans_correct')
question_questiontype(correct['question'],'question',150,'no_ans_correct')

## Pies

In [ ]:

piechart(no_ans['question_type'],labels,'no_ans')
piechart(incorrect['question_type'],labels,'no_ans_incorrect')
piechart(correct['question_type'],labels,'no_ans_correct')

In [ ]:
labels=['y','m','n']
pie_in(labels,no_ans['followup'],'no_ans','followup')
pie_in(labels,incorrect['followup'],'no_ans_incorrect','followup')
pie_in(labels,correct['followup'],'no_ans_correct','followup')

In [ ]:
no_ans['yesno'].value_counts()

# Whole set

In [ ]:
ans=pd.merge(has_ans,no_ans,how='outer')

In [ ]:
plot_histogram([(len(str(a))) for a in ans['pred_answer']], 'answer','ans')
plot_histogram([q for q in ans['question'].str.len()], 'question','ans')
plot_histogram([c for c in ans['context'].str.len()], 'context','ans')

In [ ]:
ans['ans_length']  = ans['pred_answer'].str.len()
ans['con_length']  = ans['context'].str.len()
ans['que_length']  = ans['question'].str.len()

In [ ]:
def question_questiontype(dataset,label,x,datalabel):
    answers= []
    keywords = ["why", "when", "how", "what", "who", "where",'which','other','followup_other']
    for key in keywords:
        answers.append([len(answer) for (i, answer) in enumerate(dataset) if key in ans.iloc[i]['question_type'].lower()])
    bins = np.linspace(0, x, 15)
    plt.hist(answers, bins, normed=True, label=keywords)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.legend()
    plt.title((label+' questiontype').capitalize())
    plt.savefig('graphs/'+datalabel+'_questiontype_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
question_questiontype(ans['pred_answer'],'answer',100,'ans' )
question_questiontype(ans['context'],'context',3000,'ans')
question_questiontype(ans['question'],'question',150,'ans')

In [ ]:
incorrect=ans[ans['TF'] == False]
correct=ans[ans['TF']==True]

In [ ]:
beb=ans[ans['TF'] == False]
beb.TF.value_counts()

## Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'ans_incorrect')

## Only correct answers

In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'ans_correct' )
question_questiontype(correct['context'],'context',3000,'ans_correct')
question_questiontype(correct['question'],'question',150,'ans_correct')

## Pies

In [ ]:
piechart(ans['question_type'],labels,'ans')
piechart(incorrect['question_type'],labels,'ans_incorrect')
piechart(correct['question_type'],labels,'ans_correct')

In [ ]:
labels=['y','m','n']
pie_in(labels,ans['followup'],'ans','followup')
pie_in(labels,incorrect['followup'],'ans_incorrect','followup')
pie_in(labels,correct['followup'],'ans_correct','followup')

In [ ]:
labels=['y','x','n']
pie_in(labels,ans['yesno'],'ans','yesno')
pie_in(labels,incorrect['yesno'],'ans_incorrect','yesno')
pie_in(labels,correct['yesno'],'ans_correct','yesno')

# question type analysis

In [ ]:
# crosstab
df=ans
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0, 1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/questiontype_distribution.png')
plt.show()

In [ ]:
ans.question_type.value_counts()

In [ ]:
ans[ans['question_type']=='followup_other']

# how question analysis

In [ ]:

how_analysis=pd.merge(has_ans,no_ans,how='outer')

In [ ]:
how_analysis=how_analysis.reset_index(drop=True)
how_analysis['question']=how_analysis['question'].str.lower()


In [ ]:
how_analysis.head()


In [ ]:
how_analysis['question_type']=''
qty=[]
keywords = ['how','far','long','many', 'much','old', 'come']
for index, row in how_analysis.iterrows():

    answer=row['question']
    words = answer.split()
    for key in keywords:
        if key in  words:
            qty.append(key)
    how_analysis['question_type'][index]=qty
    qty=[]



In [ ]:
how_analysis=how_analysis[how_analysis.question_type.apply(lambda x: len(x) > 1)]

In [ ]:
how_analysis.question_type.value_counts()

In [ ]:
how_analysis=how_analysis[how_analysis['question_type']!='']

In [ ]:
how_analysis['question_type'] = how_analysis['question_type'].str[1]

In [ ]:
gh=how_analysis[how_analysis['question_type']=='many']
gh.sample(6)

In [ ]:
# crosstab
df=how_analysis
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0, 1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/how_questiontype_distribution.png')
plt.show()

# followup analysis

In [ ]:
followup_analysis=pd.merge(has_ans,no_ans,how='outer')
followup_analysis.shape

In [ ]:
followup_analysis.head()

In [ ]:
followup_yes=followup_analysis[followup_analysis['followup']=='y']
followup_yes.shape

In [ ]:
followup_no=followup_analysis[followup_analysis['followup']=='n']
followup_no.shape

In [ ]:
# crosstab
df=followup_yes
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0, 1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/followup_yes_questiontype_distribution.png')
plt.show()

In [ ]:
# crosstab
df=followup_no
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0, 1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/followup_no_questiontype_distribution.png')
plt.show()

In [ ]:
followup_no.question_type.value_counts()

# ERROR ANALYSIS

# incorrectly preddicted NoAns where answer existed

In [ ]:
incorrect.loc[~incorrect.pred_answer.isin(['-']), 'pred_answer'] = 'Other'
incorrect.pred_answer.value_counts()

In [ ]:
noanswrg=incorrect[(incorrect['pred_answer'] == '-')]  
noanswrg.followup.value_counts()

In [ ]:
followup_count = noanswrg.followup.value_counts()
followup_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:
noanswrg.yesno.value_counts()

In [ ]:
yesno_count = noanswrg.yesno.value_counts()
yesno_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:

piechart(noanswrg['question_type'],labels,'')

In [ ]:
noanswrg.sample()

# question is unasnwerable but the answer was predicted

In [ ]:
incorrect.possible_answers.isnull().sum()

In [ ]:
unans=incorrect[incorrect.possible_answers.isnull()==True]  


In [ ]:
labels=['y','m','n']
pie_in(labels,unans['followup'],'unans','followup')

In [ ]:
followup_count = unans.followup.value_counts()
followup_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:
unans.yesno.value_counts()

In [ ]:
piechart(unans['question_type'],labels,'')

In [ ]:
unans.sample()

In [ ]:
incorrect=ans[ans['TF'] == False]


In [ ]:
incorrect[(incorrect['possible_answers'] == 'NaN')]  

In [ ]:
incorrect.tail(50)

In [ ]:
inc=ans[ans['TF'] == False]


In [ ]:
#dropping the previous two groups
inc=inc[~(inc['pred_answer'] == '-')]  
inc = inc[inc['possible_answers'].notna()]

In [ ]:
inc.pred_answer.value_counts()

In [ ]:
#testing if everything is gone
inc.possible_answers.isnull().sum()

# preparation for the upcoming checks whether the predicted answer contains the original answer

In [ ]:
#exploding possible answers since there may be more than one
inc['possible_answers'] = inc['possible_answers'].str.replace('[', '').astype(str)
inc['possible_answers'] = inc['possible_answers'].str.replace(']', '').astype(str)

In [ ]:
inc['possible']=inc['possible_answers']

In [ ]:
inc.head()

In [ ]:
inc= inc.assign(possible=inc['possible'].str.split('|')).explode('possible')

In [ ]:
#if the original answer is present in the predicted anser
#(that means the model picked longer sequence of words than the original answer.
#the information conveyed is still correct.)
inc['answer_text_in_prediction'] = inc.apply(lambda x: x.possible in x.pred_answer, axis=1)
inc['answer_prediction_in_answer_text'] = inc.apply(lambda x: x.pred_answer in x.possible_answers, axis=1)

In [ ]:
inc.head()

In [ ]:
#caring only about the ones that are longer
inc['poss_length']  = inc['possible'].str.len()
inc['difference'] = inc.apply(lambda x: x.poss_length < x.ans_length, axis=1)
inc['difference2'] = inc.apply(lambda x: x.poss_length > x.ans_length, axis=1)

In [ ]:
inc.head()

# predicted answer is longer than the original but still conveys correct information

In [ ]:
#grouping back after the necessary checks
inc=inc.groupby('id').agg(lambda x: x.tolist())

In [ ]:
#creating attribute to check whether if there are such instances 
inc['kkt'] = inc.apply(lambda x: any(x.answer_text_in_prediction) &  any(x.difference), axis=1)

In [ ]:
inc.kkt.value_counts()

In [ ]:
pd.set_option('display.max_colwidth', None)

# checking whether the predicted answer is shorter than the original answer but still conveys same information

In [ ]:
inc['kkt2'] = inc.apply(lambda x: any(x.answer_prediction_in_answer_text) &  any(x.difference2), axis=1)

In [ ]:
inc.kkt2.value_counts()

# since there are several possible answers for one question, its important to drop instances where some plausible answer can fulfil the condition of being longer than predicted answer and the other plausible answer for the same question fulfilling condition of beingh shorter

In [ ]:
example=inc[(inc['kkt'] == True)& (inc['kkt2'] == True)]  
example.sample()

In [ ]:
example.kkt.value_counts()

In [ ]:
last=pd.merge(incorrect,example,how='right', on='id')

In [ ]:
last.followup_x.value_counts()

In [ ]:
followup_count = last.followup_x.value_counts()
followup_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:
inc=inc[((inc['kkt'] & inc['kkt2']) == False)]  


In [ ]:
merge=pd.merge(incorrect,inc,how='right', on='id')

In [ ]:
merge.head()

In [ ]:
longer=merge[(merge['kkt'] == True)]
longer.kkt.value_counts()

In [ ]:
longer.followup_x.value_counts()

In [ ]:
followup_count = longer.followup_x.value_counts()
followup_count.plot(kind='pie',autopct='%1.0f%%')


In [ ]:
longer.yesno_x.value_counts()

In [ ]:
yesno_count = longer.yesno_x.value_counts()
yesno_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:
shorter=merge[(merge['kkt2'] == True)]  
shorter.kkt2.value_counts()

In [ ]:
shorter.followup_x.value_counts()

In [ ]:
followup_count = shorter.followup_x.value_counts()
followup_count.plot(kind='pie',autopct='%1.0f%%')


In [ ]:
shorter.yesno_x.value_counts()

In [ ]:
yesno_count = shorter.yesno_x.value_counts()
yesno_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:
piechart(shorter['question_type_x'],labels,'')

In [ ]:
piechart(longer['question_type_x'],labels,'')

In [ ]:
longer.sample()

In [ ]:
shorter.sample()

# just wrong answers

In [ ]:
wrong=merge[(merge['kkt2'] == False)&(merge['kkt'] == False)]  

In [ ]:
wrong.shape

In [ ]:
wrong.sample()

In [ ]:
piechart(wrong['question_type_x'],labels,'')

In [ ]:
followup_count = wrong.followup_x.value_counts()
followup_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:
yesno_count = wrong.yesno_x.value_counts()
yesno_count.plot(kind='pie',autopct='%1.0f%%')

In [ ]:
labels=['y','m','n']
pie_in(labels,wrong['followup_x'],'wrong','followup')

In [ ]:
wrong.sample()

# analysis of char-counts

In [ ]:
answ=ans
def analyse_lowest(part, label):
    se=100000
    for q in part.str.len():
        if se>q:
            se=q
            
    print('The length of ',label,'starts at ',se)
def analyse_highest(part, label):
    se=0
    for q in part.str.len():
        if se<q:
            se=q
            
    print('The length of ',label,'ends at ',se)
answ = answ[answ['possible_answers'].notna()]
#exploding possible answers
answ['possible_answers'] = answ['possible_answers'].str.replace('[', '').astype(str)
answ['possible_answers'] = answ['possible_answers'].str.replace(']', '').astype(str)
answ['possible']=answ['possible_answers']    
answ= answ.assign(possible=answ['possible'].str.split('|')).explode('possible')

In [ ]:
analyse_lowest(answ['possible'],'original answer')
analyse_lowest(answ['context'],'context')
analyse_lowest(answ['question'],'question')
analyse_lowest(answ['pred_answer'],'pred_answer')
analyse_highest(answ['possible'],'original answer')
analyse_highest(answ['context'],'context')
analyse_highest(answ['question'],'question')
analyse_highest(answ['pred_answer'],'pred_answer')

In [ ]:

ans['question_type']=''
qty=[]
keywords = ["why", "when", "how", "what", "who", "where",'which', 'whom','whose','does']
for index, row in ans.iterrows():

    answer=row['question']
    words = answer.split()
    for key in keywords:
        if key in  words:
            qty.append(key)
    ans['question_type'][index]=qty
    qty=[]
ans.question_type.value_counts()
